In [3]:
import pandas as pd
import numpy as np

In [1]:
!pip install tensorflow

In [6]:
data=pd.read_csv("IMDB Dataset.csv")

In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
data.shape

(50000, 2)

In [9]:
type(data)


pandas.core.frame.DataFrame

In [10]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [13]:
#target converted to number
#one hot encoding
#label encoding

#positive -1
#negatice -0




In [11]:
data.replace({"sentiment":{"positive":1 , "negative" :0}},inplace=True)

C:\Users\sanoj\AppData\Local\Temp\ipykernel_12768\1310870886.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1 , "negative" :0}},inplace=True)


In [12]:
 data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [19]:
#lSTM -LONG SHORT TERM MEMORY

##RNN

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
train_data,test_data = train_test_split(data , test_size=0.2 ,random_state=42)

In [15]:
train_data.shape

(40000, 2)

In [16]:
test_data.shape

(10000, 2)

In [17]:
##tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

In [18]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [19]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [20]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [21]:
y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

####################LSTM MODELBUILDING #######################

In [22]:
model = Sequential()
model.add(Embedding(input_dim=5000 ,output_dim =128 , input_length=200))
model.add(LSTM(128 , dropout=0.2 , recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))   ##binary classification


c:\Users\sanoj\IMDB-Movie-Review-Sentiment-Analysis-using-RNN-and-TensorFlow\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


##model.summary()

In [23]:
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer = "adam" , loss="binary_crossentropy" ,metrics=["accuracy"])

In [25]:
model.fit(X_train , y_train ,epochs=5 , batch_size = 64 , validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 386s 761ms/step - accuracy: 0.7187 - loss: 0.5416 - val_accuracy: 0.8441 - val_loss: 0.3811
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 338s 677ms/step - accuracy: 0.8505 - loss: 0.3557 - val_accuracy: 0.8355 - val_loss: 0.4503
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 345s 689ms/step - accuracy: 0.8699 - loss: 0.3238 - val_accuracy: 0.8612 - val_loss: 0.3363
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 390s 780ms/step - accuracy: 0.8928 - loss: 0.2727 - val_accuracy: 0.8554 - val_loss: 0.3820
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 349s 698ms/step - accuracy: 0.9080 - loss: 0.2384 - val_accuracy: 0.8681 - val_loss: 0.3196


In [26]:
model.save('model.h5')

In [27]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [28]:
loss,accuracy = model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 52ms/step - accuracy: 0.8698 - loss: 0.3045


In [29]:
print(accuracy)

0.8712000250816345


In [151]:
##building predictive system


In [30]:
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequences = pad_sequences(sequences ,maxlen =200)
    prediction = model.predict(padded_sequences)
    sentiment = "POSTIVE" if prediction[0][0] > 0.5 else "NEGATIVE"
    #print(prediction[0][0])
    return sentiment

In [31]:
predictive_system("the movie is blockbuster")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'POSTIVE'

In [38]:
predictive_system("the movie funny something predictable")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


'NEGATIVE'